In [3]:
import pandas as pd
import numpy as np


def read_commission_data(file_name):
    """
    read commission data from excel file
        parameter:
            file_name: str, the path of the excel file
        return:
            df: pd.DataFrame, the commission data
    """
    return pd.read_excel(file_name)

def extract_commission_period(file_name):
    """
    extract commission period from file name
        parameter:
            file_name: str, the path of the excel file
        return:
            commission_period: str, the commission period
    """
    return file_name.split(' ')[1]

def normalize_enblem_data(enblem):
    """
    normalize emblem data, calculate enrollment_type and add member_name column
        parameter:
            enblem: pd.DataFrame, the emblem data
        return:
            enblem: pd.DataFrame, the normalized emblem data
    """
    enblem['enrollment_type'] = enblem.apply(lambda x: 'Renewal' if x['Prior Plan'] == 'Yes' else 'Initial', axis=1)
    enblem['member_name'] = enblem['Member First Name'] + ' ' + enblem['Member Last Name']
    return enblem

def rename_columns(df, column_map):
    """
    rename columns according to the mapping
        parameter:
            df: pd.DataFrame, the data
            column_map: dict, the column mapping
        return:
            df: pd.DataFrame, the data with renamed columns
    """
    return df.rename(columns=column_map)

def add_carrier_and_period(df, carrier, commission_period):
    """
    add carrier and commission_period columns
        parameter:
            df: pd.DataFrame, the data
            carrier: str, the carrier
            commission_period: str, the commission period
        return:
            df: pd.DataFrame, the data with added columns
    """
    df['carrier'] = carrier
    df['commission_period'] = commission_period
    return df

def clean_data(df):
    """
    data cleaning, format date and standardize agency_name
        parameter:
            df: pd.DataFrame, the data
        return:
            df: pd.DataFrame, the cleaned data
    """
    df_clean = (
        df
        .assign(pay_period=lambda x: pd.to_datetime(x['pay_period'], format='%m/%Y', errors='coerce').astype(str))
        .assign(pay_period=lambda x: x['pay_period'].str[:7])
        .assign(agency_name=lambda x: np.where(x['agency_name'].str.contains('Delta', na=False), 'Delta Care', x['agency_name']))
        .assign(signed_date=lambda x: pd.to_datetime(x['signed_date'], errors='coerce'))
        .assign(effective_date=lambda x: pd.to_datetime(x['effective_date'], errors='coerce'))
    )
    return df_clean

def get_top_agents(df, commission_period, agency_name, top_n=10):
    """
    calculate the total commission of the top N agents for the specified commission period and agency
        parameter:
            df: pd.DataFrame, the data
            commission_period: str, the commission period
            agency_name: str, the agency name
            top_n: int, the number of top agents
        return:
            top_agents: pd.DataFrame, the top agents
    """
    top_agents = (
        df
        .loc[lambda x: x['commission_period'] == commission_period]
        .loc[lambda x: x['agency_name'] == agency_name]
        .groupby('agent_name')
        .agg(total_commission=('payment_amount', 'sum'))
        .sort_values(by='total_commission', ascending=False)
        .reset_index()
        .head(top_n)
    )
    return top_agents

def parse_commission_data(file_names_and_mappings):
    """
    parse commission data from multiple files and merge
        parameter:
            file_names_and_mappings: list, the file names and mappings
        return:
            df: pd.DataFrame, the merged data
    """
    data_frames = []
    for file_name, column_map, carrier in file_names_and_mappings:
        df = read_commission_data(file_name)
        commission_period = extract_commission_period(file_name)
        if carrier == 'Emblem':
            df = normalize_enblem_data(df)
        df = rename_columns(df, column_map)
        df = add_carrier_and_period(df, carrier, commission_period)
        data_frames.append(df)
    return pd.concat(data_frames, ignore_index=True)

def save_to_csv(df, file_name):
    """
    save cleaned data to csv file
        parameter:
            df: pd.DataFrame, the cleaned data
            file_name: str, the path of the csv file
    """
    df.to_csv(file_name, index=False)

# ----------------------------------------------------------------------------------------------------------------------

# define file names and column mappings

# file names and column mappings
file_names_and_mappings = [
    ('./Centene 06.2024 Commission.xlsx', {
        'Payment Type': 'enrollment_type',
        'Payment Description': 'enrollment_description',
        'Writing Broker NPN': 'agent_id',
        'Writing Broker Name': 'agent_name',
        'Earner NPN': 'agency_id',
        'Earner Name': 'agency_name',
        'Pay Period': 'pay_period',
        'Payment Amount': 'payment_amount',
        'Medicare Beneficiary Identifier (MBI)': 'medicare_beneficiary_identifier',
        'Member Name': 'member_name',
        'Signed Date': 'signed_date',
        'Effective Date': 'effective_date',
        'Cycle Year': 'cycle_year',
        'Prior Plan Type': 'prior_plan_type',
        'Policy State': 'policy_state',
        'Plan Type': 'plan_type',
        'Plan Name': 'plan_name',
        'Original Effective Date': 'original_effective_date',
        'Member Term Date': 'term_date',
        'CMS Contract': 'cms_contract',
        'PBP': 'pbp',
        'Description': 'description',
        'Centene ID': 'centene_id',
        'QIS Level': 'qis_level'
    }, 'Centene'),
    ('./Emblem 06.2024 Commission.xlsx', {
        'Payee Name': 'agency_name',
        'Payee ID': 'agency_id',
        'Rep Name': 'agent_name',
        'Rep ID': 'agent_id',
        'Effective Date': 'effective_date',
        'Term Date': 'term_date',
        'Plan Group': 'plan_group',
        'Plan Group Name': 'plan_group_name',
        'Plan': 'plan_name',
        'OSB': 'osb',
        'Member HIC': 'member_hic',
        'Member ID': 'member_id',
        'Member First Name': 'member_first_name',
        'Member Last Name': 'member_last_name',
        'Member Year': 'member_year',
        'Cycle Year': 'cycle_year',
        'Prior Plan': 'prior_plan_type',
        'Payment': 'payment_amount'
    }, 'Emblem'),
    ('./Healthfirst 06.2024 Commission.xlsx', {
        'Member ID': 'member_id',
        'Member Name': 'member_name',
        'Member Effective Date': 'effective_date',
        'Period': 'pay_period',
        'Product': 'plan_name',
        'Chargeback': 'chargeback',
        'Enrollment Type': 'enrollment_type',
        'Producer Type': 'agency_type',
        'Producer Name': 'agency_name',
        'Disenrollement Reason': 'term_reason',
        'Disenrolled Date': 'term_date',
        'Adjustment Description': 'adjustment_description',
        'Amount': 'payment_amount'
    }, 'Healthfirst')
]

# parse and clean data
commission_data = parse_commission_data(file_names_and_mappings)
cleaned_data = clean_data(commission_data)

# save to csv file
save_to_csv(cleaned_data, 'commission_data.csv')

# get top 10 agents
top_10_agents = get_top_agents(cleaned_data, '06.2024', 'Delta Care')
print(top_10_agents)

          agent_name  total_commission
0   Jeffrey Anderson           2250.59
1        Kevin Parks           2058.12
2  James Martinez Jr           1684.04
3        Eric Wilson           1091.84
4     Chelsea Butler            969.29
5       Alexis Huang            917.19
6         Kirk Baker            883.40
7      Todd Martinez            875.16
8   Jennifer Jackson            715.00
9        Diana Weber            605.43
